In [1]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from huggingface_hub import notebook_login
from datasets import load_dataset
torch.cuda.empty_cache()

## 1. 허깅페이스 로그인

In [2]:
notebook_login()

## 2. 데이터셋 로드

In [3]:
# TEST_NAME = "beomi-gemma-ko-2b-01"
# TEST_NAME = "google-gemma-ko-2b-01"
# TEST_NAME = "beomi-gemma-ko-7b"
# TEST_NAME = "beomi-gemma-mling-7b"
# TEST_NAME = "upstage-SOLAR-10.7B-Instruct-v1.0"
# TEST_NAME = "beomi-OPEN-SOLAR-KO-10.7B"
TEST_NAME = "yanolja-EEVE-Korean-10.8B-v1.0"

import os
import platform

# 데이터 관련
os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'
DATA_PATH = PRE_PATH + "data/" # 대회에서 제공한 데이터
TRAIN_PATH = DATA_PATH + "train_kkh_new.csv"
VALID_PATH = DATA_PATH + "dev_kkh_new.csv"
TEST_PATH = DATA_PATH + "test.csv"
PREDICTION_PATH = PRE_PATH + "prediction/" # 최종 예측 값
SUBMIT_PATH = PREDICTION_PATH + "submission_" + TEST_NAME + ".csv"

# BASE_MODEL = "google/gemma-2b-it"
# BASE_MODEL = "beomi/gemma-ko-2b"
# BASE_MODEL = "beomi/gemma-ko-7b"
# BASE_MODEL = "beomi/gemma-mling-7b"
# BASE_MODEL = "upstage/SOLAR-10.7B-Instruct-v1.0"
# BASE_MODEL = "beomi/OPEN-SOLAR-KO-10.7B"
BASE_MODEL = "yanolja/EEVE-Korean-10.8B-v1.0"
FINETUNE_MODEL = PRE_PATH + f"gemma_model/{TEST_NAME}"

DATASET_TRAIN = load_dataset('csv', data_files={'train': TRAIN_PATH})
DATASET_TEST = load_dataset('csv', data_files={'test': TEST_PATH})
TRAIN_DATA = DATASET_TRAIN['train']

In [4]:
# 데이터셋 확인
print(TRAIN_DATA[0])
print('=============================')
print(f"train 대화문:\n{TRAIN_DATA[0]['dialogue']}")
print('=============================')
print(f"train 요약문:\n{TRAIN_DATA[0]['summary']}")
print('=============================')
print(f"test 대화문:\n{TRAIN_DATA[0]['dialogue']}")

{'fname': 'train_0', 'dialogue': '#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?\n#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.\n#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.\n#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?\n#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.\n#Person2#: 알겠습니다.\n#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?\n#Person2#: 네.\n#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. \n#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.\n#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.\n#Person2#: 알겠습니다, 감사합니다, 의사선생님.', 'summary': '스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.', 'topic': '건강검진 받기'}
train 대화문:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 

## 3.1 모델 로드

In [5]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

In [6]:
prompt = "한국의 아이돌 문화에 대해 알려줘."

# 텍스트 생성을 위한 파이프라인 설정
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512) # max_new_tokens: 생성할 최대 토큰 수
outputs = pipe(
    prompt,
    do_sample=True, # 샘플링 전략 사용. 확률 분포를 기반으로 다음 토큰을 선택
    temperature=0.2, # 샘플링의 다양성을 조절하는 파라미터. 값이 높을수록 랜덤성 증가
    top_k=50, # 다음 토큰을 선택할 때 상위 k개의 후보 토큰 중에서 선택. 여기에서는 상위 50개의 후보 토큰 중에서 샘플링
    top_p=0.95, # 누적 확률이 p가 될 때까지 후보 토큰을 포함
    repetition_penalty=1.2, # 반복 패널티를 적용하여 같은 단어나 구절이 반복되는 것 방지
    add_special_tokens=True # 모델이 입력 프롬프트의 시작과 끝을 명확히 인식할 수 있도록 특별 토큰 추가
)
print(outputs[0]["generated_text"][len(prompt):]) # 입력 프롬프트 이후에 생성된 텍스트만 출력

/opt/conda/envs/nlp/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


 | 1boon
한국의 아이돌 문화에 대해 알려줘.
아이돌은 어떻게 만들어지는가?
<한겨레> 김태권 기자는 <아이돌, 한국 대중문화의 진화>(2016)에서 "아이돌 그룹이 탄생하는 과정은 '제조'라는 단어로는 부족하다"고 말한다. 제조란 단순히 상품화 과정을 뜻하는데, 이보다 더 복잡한 과정이 존재한다는 것이다.
그 첫 번째 단계로 그는 '발견'을 제시한다. 가요계에서는 이를 '스카우트(scout)'라고 부른다. 스카우트는 연예기획사나 기획사가 발굴해낸 유망주들을 모아놓은 일종의 데이터베이스를 의미한다. 여기에는 노래와 무대 실력뿐 아니라 외모, 성격 등 다양한 정보가 담겨 있다.
두 번째는 바로 '육성'이다. 육성이란 말 그대로 어린 연습생들이 데뷔할 수 있는 수준까지 성장하도록 도와주는 과정인데, 이때 가장 중요한 것은 트레이닝 시스템이라고 할 수 있다.
세 번째로 소개된 것이 바로 '마케팅'이다. 마케팅을 통해 소비자들에게 인지도를 심어주고, 나아가 충성스러운 소비자층으로 만드는 게 중요하다.


## 4.1 학습용 프롬프트 조정

In [13]:
# NF4 양자화를 위한 설정
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True, # 모델을 4비트 정밀도로 로드
    bnb_4bit_quant_type="nf4", # 4비트 NormalFloat 양자화: 양자화된 파라미터의 분포 범위를 정규분포 내로 억제하여 정밀도 저하 방지
    bnb_4bit_use_double_quant=True, # 이중 양자화: 양자화를 적용하는 정수에 대해서도 양자화 적용
    bnb_4bit_compute_dtype=torch.bfloat16 # 연산 속도를 높이기 위해 사용 (default: torch.float32)
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=nf4_config,
    device_map="auto"
)

/opt/conda/envs/nlp/lib/python3.10/site-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 1568.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [14]:
prompt_input_template = """아래는 작업을 설명하는 지시사항과 추가 정보를 제공하는 입력이 짝으로 구성됩니다. 이에 대한 적절한 응답을 작성해주세요.

### 지시사항:
{instruction}

### 입력:
{input}

### 응답:"""


prompt_no_input_template = """아래는 작업을 설명하는 지시사항입니다. 이에 대한 적절한 응답을 작성해주세요.

### 지시사항:
{instruction}

### 응답:"""

In [15]:
def generate_prompt(data_point):
  instruction = data_point["instruction"]
  input = data_point["input"]
  label = data_point["output"]

  if input:
    res = prompt_input_template.format(instruction=instruction, input=input)
  else:
    res = prompt_no_input_template.format(instruction=instruction)

  if label:
    res = f"{res}{label}<|im_end|>" # eos_token을 마지막에 추가

  data_point['text'] = res

  return data_point

In [18]:
# 데이터셋에 프롬프트 적용
remove_column_keys = TRAIN_DATASET.features.keys() # 기존 컬럼(instruction, output 등) 제거
dataset_cvted = TRAIN_DATASET.shuffle().map(generate_prompt, remove_columns=remove_column_keys)

NameError: name 'TRAIN_DATASET' is not defined

In [ ]:
def tokenize_function(examples):
  outputs = tokenizer(examples["text"], truncation=True, max_length=512)
  return outputs
remove_column_keys = dataset_cvted.features.keys()
dataset_tokenized = dataset_cvted.map(tokenize_function, batched=True, remove_columns=remove_column_keys)

## 4.2 QLoRA 설정

In [9]:
lora_config = LoraConfig(
    r=4, # LoRA 가중치 행렬의 rank. 정수형이며 값이 작을수록 trainable parameter가 적어짐
    lora_alpha=8, # LoRA 스케일링 팩터. 추론 시 PLM weight와 합칠 때 LoRA weight의 스케일을 일정하게 유지하기 위해 사용
    lora_dropout=0.05,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], # LoRA를 적용할 layer. 모델 아키텍처에 따라 달라짐
    bias='none', # bias 파라미터를 학습시킬지 지정. ['none', 'all', 'lora_only']
    task_type=TaskType.CAUSAL_LM
)

# 양자화된 모델을 학습하기 전, 전처리를 위해 호출
model = prepare_model_for_kbit_training(model)
# LoRA 학습을 위해서는 아래와 같이 peft를 사용하여 모델을 wrapping 해주어야 함
model = get_peft_model(model, lora_config)

# 학습 파라미터 확인
model.print_trainable_parameters()

In [10]:
# Data Collator 역할
# 각 입력 시퀀스의 input_ids(토큰) 길이를 계산하고, 가장 긴 길이를 기준으로 길이가 짧은 시퀀스에는 패딩 토큰 추가
def collate_fn(examples):
    examples_batch = tokenizer.pad(examples, padding='longest', return_tensors='pt')
    examples_batch['labels'] = examples_batch['input_ids'] # 모델 학습 평가를 위한 loss 계산을 위해 입력 토큰을 레이블로 사용
    return examples_batch

train_args = transformers.TrainingArguments(
    per_device_train_batch_size=2, # 각 디바이스당 배치 사이즈. 작을수록(1~2) 좀 더 빠르게 alignment 됨
    gradient_accumulation_steps=4, 
    warmup_steps=1,
    #num_train_epochs=1,
    max_steps=1000, 
    learning_rate=2e-4, # 학습률
    bf16=True, # bf16 사용 (지원되는 하드웨어 확인 필요)
    output_dir="outputs",
    optim="paged_adamw_8bit", # 8비트 AdamW 옵티마이저
    logging_steps=50, # 로깅 빈도
    save_total_limit=3 # 저장할 체크포인트의 최대 수
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_tokenized,
    max_seq_length=512, # 최대 시퀀스 길이
    args=train_args,
    dataset_text_field="text",
    data_collator=collate_fn
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
model.config.use_cache = False
trainer.train()

## 4.3 Trainer 실행

In [11]:
trainer = SFTTrainer(
    model=model,
    train_dataset=TRAIN_DATA,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 1,
        max_steps=100,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_ratio=0.02,
        learning_rate=2e-4,
        fp16=True,
        fp16_full_eval=True,  # kkh 추가함
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

Map:   0%|          | 0/12448 [00:00<?, ? examples/s]

/opt/conda/envs/nlp/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
max_steps is given, it will override any value given in num_train_epochs


In [12]:
trainer.train()

Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 

## 4.4 Finetuned Model 저장

In [12]:
ADAPTER_MODEL = "lora_adapter"
trainer.model.save_pretrained(ADAPTER_MODEL)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained(FINETUNE_MODEL)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Some parameters are on the meta device because they were offloaded to the cpu.
/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/modeling_utils.py:2633: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## 5.1 Fine-tuned 모델 로드

In [13]:
finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 

## 5.2 Fine-tuned 모델 추론

In [18]:
# ex = DATASET_TEST['test']['dialogue'][0]
ex = DATASET_TRAIN['train']['dialogue'][0]
messages = [
    {
        "role": "user",
        "content": "다음 글을 요약해주세요:\n\n{}".format(ex)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [19]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

#Person2#는 스미스씨에게 건강검진을 받고 싶다고 말합니다. #Person1#은 스미스씨에게 도움이 될 수 있는 정보와 약물을 제공하고 있습니다.


